# Predicting Sale Prices for Ames Iowa Housing Dataset

#### By: _Noah C. (DSI)_

## Import Libraries & Load in Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [3]:
### IMPORTING ALREADY CLEANED NOTEBOOKS ###

df_train = pd.read_csv("./clean_train.csv")
df_test  = pd.read_csv("./clean_test.csv")

In [4]:
df_train.shape

(2051, 40)

In [5]:
df_test.shape

(878, 39)

In [6]:
## What column in train is not in test?
set(df_train) - set(df_test)

{'SalePrice'}

In [7]:
df_train.head()


,Id,PID,MS Zoning,Street,Alley,Land Contour,Lot Config,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Foundation,Bsmt Qual,Total Bsmt SF,Heating,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,TotRms AbvGrd,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Misc Feature,Sale Type,SalePrice
0,109,533352170,RL,Pave,0,Lvl,CulDSac,Sawyer,RRAe,Norm,1Fam,2Story,6,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,4,CBlock,3,725.0,GasA,725,1479,2,4,6,0,Attchd,1976.0,2,2.0,475.0,0,WD,130500
1,544,531379050,RL,Pave,0,Lvl,CulDSac,SawyerW,Norm,Norm,1Fam,2Story,7,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,4,PConc,4,913.0,GasA,913,2122,2,4,8,3,Attchd,1997.0,2,2.0,559.0,0,WD,220000
2,153,535304180,RL,Pave,0,Lvl,Inside,NAmes,Norm,Norm,1Fam,1Story,5,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,CBlock,3,1057.0,GasA,1057,1057,1,4,5,0,Detchd,1953.0,1,1.0,246.0,0,WD,109000
3,318,916386060,RL,Pave,0,Lvl,Inside,Timber,Norm,Norm,1Fam,2Story,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,PConc,4,384.0,GasA,744,1444,2,3,7,0,BuiltIn,2007.0,3,2.0,400.0,0,WD,174000
4,255,906425045,RL,Pave,0,Lvl,Inside,SawyerW,Norm,Norm,1Fam,1.5Fin,6,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,3,PConc,2,676.0,GasA,831,1445,2,3,6,0,Detchd,1957.0,1,2.0,484.0,0,WD,138500


In [9]:
df_train.set_index('Id', inplace = True)
df_test.set_index('Id', inplace = True)

## Data Cleaning

**Set `Id` column to be our new index for both train and test datasets.**

This will make submitting to Kaggle easier later on. It also lets us reference rows with the `Id` number when using `.loc`.

In [10]:
df_train.head()

,PID,MS Zoning,Street,Alley,Land Contour,Lot Config,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Foundation,Bsmt Qual,Total Bsmt SF,Heating,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,TotRms AbvGrd,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Misc Feature,Sale Type,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
109,533352170,RL,Pave,0,Lvl,CulDSac,Sawyer,RRAe,Norm,1Fam,2Story,6,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,4,CBlock,3,725.0,GasA,725,1479,2,4,6,0,Attchd,1976.0,2,2.0,475.0,0,WD,130500
544,531379050,RL,Pave,0,Lvl,CulDSac,SawyerW,Norm,Norm,1Fam,2Story,7,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,4,PConc,4,913.0,GasA,913,2122,2,4,8,3,Attchd,1997.0,2,2.0,559.0,0,WD,220000
153,535304180,RL,Pave,0,Lvl,Inside,NAmes,Norm,Norm,1Fam,1Story,5,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,CBlock,3,1057.0,GasA,1057,1057,1,4,5,0,Detchd,1953.0,1,1.0,246.0,0,WD,109000
318,916386060,RL,Pave,0,Lvl,Inside,Timber,Norm,Norm,1Fam,2Story,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,PConc,4,384.0,GasA,744,1444,2,3,7,0,BuiltIn,2007.0,3,2.0,400.0,0,WD,174000
255,906425045,RL,Pave,0,Lvl,Inside,SawyerW,Norm,Norm,1Fam,1.5Fin,6,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,3,PConc,2,676.0,GasA,831,1445,2,3,6,0,Detchd,1957.0,1,2.0,484.0,0,WD,138500


Check to see how many missing values are in each column.

In [ ]:
df_test.isnull().sum().sort_values(ascending = False)

In [ ]:
# ### EXAMPLE ONLY - DO NOT DO THIS IN YOUR OWN PROJECT!
# ### Lazy workflow - I'm going to fill all NAs with 0 (again, do not do this)

# df_train.fillna(0, inplace = True)
# df_test.fillna(0, inplace = True)

In [11]:
df_train = pd.get_dummies(df_train, columns = ['MS Zoning', 'Lot Config', 'House Style','Garage Type'])
df_test = pd.get_dummies(df_test, columns = ['MS Zoning', 'Lot Config', 'House Style','Garage Type'])

In [12]:
df_train.head()

,PID,Street,Alley,Land Contour,Neighborhood,Condition 1,Condition 2,Bldg Type,Overall Qual,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Foundation,Bsmt Qual,Total Bsmt SF,Heating,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,TotRms AbvGrd,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Misc Feature,Sale Type,SalePrice,MS Zoning_A (agr),MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Lot Config_Corner,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Garage Type_0,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
109,533352170,Pave,0,Lvl,Sawyer,RRAe,Norm,1Fam,6,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,4,CBlock,3,725.0,GasA,725,1479,2,4,6,0,1976.0,2,2.0,475.0,0,WD,130500,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
544,531379050,Pave,0,Lvl,SawyerW,Norm,Norm,1Fam,7,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,4,PConc,4,913.0,GasA,913,2122,2,4,8,3,1997.0,2,2.0,559.0,0,WD,220000,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
153,535304180,Pave,0,Lvl,NAmes,Norm,Norm,1Fam,5,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,CBlock,3,1057.0,GasA,1057,1057,1,4,5,0,1953.0,1,1.0,246.0,0,WD,109000,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
318,916386060,Pave,0,Lvl,Timber,Norm,Norm,1Fam,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,PConc,4,384.0,GasA,744,1444,2,3,7,0,2007.0,3,2.0,400.0,0,WD,174000,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
255,906425045,Pave,0,Lvl,SawyerW,Norm,Norm,1Fam,6,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,3,PConc,2,676.0,GasA,831,1445,2,3,6,0,1957.0,1,2.0,484.0,0,WD,138500,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [13]:
df_test.head()

,PID,Street,Alley,Land Contour,Neighborhood,Condition 1,Condition 2,Bldg Type,Overall Qual,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Foundation,Bsmt Qual,Total Bsmt SF,Heating,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,TotRms AbvGrd,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Misc Feature,Sale Type,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Lot Config_Corner,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Garage Type_0,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2658,902301120,Pave,Grvl,Lvl,OldTown,Norm,Norm,2fmCon,6,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,3,Stone,2,1020,GasA,908,1928,2,2,9,0,1910.0,1,1,440,0,WD,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2718,905108090,Pave,0,Lvl,Sawyer,Norm,Norm,Duplex,5,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,3,CBlock,4,1967,GasA,1967,1967,2,3,10,0,1977.0,3,2,580,0,WD,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2414,528218130,Pave,0,Lvl,Gilbert,Norm,Norm,1Fam,7,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,4,PConc,4,654,GasA,664,1496,2,4,7,4,2006.0,2,2,426,0,New,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1989,902207150,Pave,0,Lvl,OldTown,Norm,Norm,1Fam,5,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,4,CBlock,3,968,GasA,968,968,1,3,5,0,1935.0,1,2,480,0,WD,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
625,535105100,Pave,0,Lvl,NAmes,Norm,Norm,1Fam,6,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,3,CBlock,4,1394,GasA,1394,1394,1,3,6,4,1963.0,2,2,514,0,WD,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [14]:
df_train.shape

(2051, 62)

In [15]:
df_test.shape

(878, 60)

In [16]:
df_train.drop([ 'MS Zoning_RH', 'Lot Config_Corner', 'House Style_SFoyer', 'Garage Type_Basment'], axis=1, inplace=True)
df_test.drop([ 'MS Zoning_RH', 'Lot Config_Corner', 'House Style_SFoyer', 'Garage Type_Basment'], axis=1, inplace=True)

In [17]:
df_train.shape

(2051, 58)

In [18]:
df_test.shape

(878, 56)

In [19]:
df_test.head()

,PID,Street,Alley,Land Contour,Neighborhood,Condition 1,Condition 2,Bldg Type,Overall Qual,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Foundation,Bsmt Qual,Total Bsmt SF,Heating,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,TotRms AbvGrd,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Misc Feature,Sale Type,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RL,MS Zoning_RM,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SLvl,Garage Type_0,Garage Type_2Types,Garage Type_Attchd,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2658,902301120,Pave,Grvl,Lvl,OldTown,Norm,Norm,2fmCon,6,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,3,Stone,2,1020,GasA,908,1928,2,2,9,0,1910.0,1,1,440,0,WD,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1
2718,905108090,Pave,0,Lvl,Sawyer,Norm,Norm,Duplex,5,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,3,CBlock,4,1967,GasA,1967,1967,2,3,10,0,1977.0,3,2,580,0,WD,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0
2414,528218130,Pave,0,Lvl,Gilbert,Norm,Norm,1Fam,7,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,4,PConc,4,654,GasA,664,1496,2,4,7,4,2006.0,2,2,426,0,New,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
1989,902207150,Pave,0,Lvl,OldTown,Norm,Norm,1Fam,5,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,4,CBlock,3,968,GasA,968,968,1,3,5,0,1935.0,1,2,480,0,WD,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
625,535105100,Pave,0,Lvl,NAmes,Norm,Norm,1Fam,6,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,3,CBlock,4,1394,GasA,1394,1394,1,3,6,4,1963.0,2,2,514,0,WD,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0


In [20]:
df_train.head()

,PID,Street,Alley,Land Contour,Neighborhood,Condition 1,Condition 2,Bldg Type,Overall Qual,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Foundation,Bsmt Qual,Total Bsmt SF,Heating,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,TotRms AbvGrd,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Misc Feature,Sale Type,SalePrice,MS Zoning_A (agr),MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RL,MS Zoning_RM,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SLvl,Garage Type_0,Garage Type_2Types,Garage Type_Attchd,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
109,533352170,Pave,0,Lvl,Sawyer,RRAe,Norm,1Fam,6,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,4,CBlock,3,725.0,GasA,725,1479,2,4,6,0,1976.0,2,2.0,475.0,0,WD,130500,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
544,531379050,Pave,0,Lvl,SawyerW,Norm,Norm,1Fam,7,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,4,PConc,4,913.0,GasA,913,2122,2,4,8,3,1997.0,2,2.0,559.0,0,WD,220000,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
153,535304180,Pave,0,Lvl,NAmes,Norm,Norm,1Fam,5,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,CBlock,3,1057.0,GasA,1057,1057,1,4,5,0,1953.0,1,1.0,246.0,0,WD,109000,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
318,916386060,Pave,0,Lvl,Timber,Norm,Norm,1Fam,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,PConc,4,384.0,GasA,744,1444,2,3,7,0,2007.0,3,2.0,400.0,0,WD,174000,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
255,906425045,Pave,0,Lvl,SawyerW,Norm,Norm,1Fam,6,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,3,PConc,2,676.0,GasA,831,1445,2,3,6,0,1957.0,1,2.0,484.0,0,WD,138500,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1


In [21]:
set(df_train) - set(df_test)

{'MS Zoning_A (agr)', 'SalePrice'}

In [22]:
df_train.drop(['MS Zoning_A (agr)'], axis=1, inplace=True)

In [24]:
df_train.shape

(2051, 57)

In [43]:
df_test.shape

(878, 56)

## EDA (Exploratory Data Analysis)
I'm going to skip this step right now, but you should not do so in your project.

Things that go here:
- descriptive statistics
- visualizations
- visualizations
- visualizations
- interpretations of descriptive statistics and visualizations

## Feature Engineering

In [ ]:
# ## one-hot encode the values in the Neighborhood column for both the training and testing datasets
# train_dummies = pd.get_dummies(df_train['Neighborhood'])
# test_dummies  = pd.get_dummies(df_test['Neighborhood'])

In [ ]:
# ## Find the columns that are in test, but are not in train
# missing_columns_train = list(set(test_dummies) - set(train_dummies))

# ## Find the columns that are in train, but are not in test
# missing_columns_test  = list(set(train_dummies) - set(test_dummies))

# print(f"Columns missing from Train: {missing_columns_train}")
# print(f"Columns missing from Test: {missing_columns_test}")

In [ ]:
# ## Add the missing columns to the test_dummies dataframe
# ### Since no rows in the test dataset have values for these new columns, fill them with 0's
# for col in missing_columns_test:
#     test_dummies[col] = 0

In [ ]:
# ## Check to see that it worked.
# set(train_dummies) - set(test_dummies)

In [ ]:
# ## Take a look at all the dummy columns created
# train_dummies.columns

Since we want to do `drop_first = True`, we'll manually drop the `Blmngtn` column from both dummy dataframes.

- I chose `Blmngtn` to be my reference category simply because it was the first alphabetically.

- However, this also means that when I interpret my coefficients later on, I'll want to keep in mind that a zero in every Neighborhood dummy column means the row had a house in `Blmngtn`. So the effect of the `Blmngtn` column is now captured in the intercept, and all of my other coefficients will be interpreted in comparison to a house in `Blmngtn`. (e.g. "for a house in Old Town **compared to** a house in Bloomington").

In [ ]:
# ## Drop one of the neighborhood columns from both train and test
# train_dummies.drop(columns = ['Blmngtn'], inplace = True)
# test_dummies.drop(columns = ['Blmngtn'], inplace = True)

In [ ]:
# ## Join the train and test dummy columns back with the original dataframes
# df_train_full = df_train.join(train_dummies)
# df_test_full  = df_test.join(test_dummies)

# ## Make sure the columns in `df_test_full` are in the same order as the column in `df_train_full`
# df_test_full = df_test_full[df_train_full.drop(columns = ['SalePrice']).columns]

In [ ]:
df_train_full.columns

In [ ]:
df_test_full.columns

## Preprocessing & Modeling

In [51]:
# ## Define `features` list of features to include in your model
features = df_train.columns.drop(['SalePrice', 'Neighborhood', 'Street', 'Alley', 'Land Contour', 'Condition 1', 'Condition 2', 'Bldg Type', 'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Foundation', 'Heating', 'Misc Feature', 'Sale Type'])

# ## We use the list `.extend()` method here instead of `.append()` in order to add
# ## all of the elements of the list individually, instead of adding them all as one list.
# features.extend(train_dummies.columns)

# print(features)

In [54]:
X = df_train[features]
y = df_train['SalePrice']

# ## Define X and y
# X = df_train_full[features]
# y = df_train_full['SalePrice']

In [55]:
## Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [56]:
## Take a look at the shapes of X_train and X_test
print(X_train.shape)

print(X_test.shape)

(1538, 40)
(513, 40)


In [57]:
## Establish a baseline model for comparison
### We can think of our baseline model as a naive model that always
### predicts the mean of our target
y_train_baseline = [y_train.mean()]*len(y_train)        ## multiplying a list by an integer dupicates the elements of that list that many times.
y_test_baseline  = [y_train.mean()]*len(y_test)

In [58]:
## Calculate the RMSE for the baseline train and test "predictions"
print(f"Baseline RMSE - Train: {mean_squared_error(y_train, y_train_baseline)**0.5}")
print(f"Baseline RMSE - Test: {mean_squared_error(y_test, y_test_baseline)**0.5}")

Baseline RMSE - Train: 79526.85223710592
Baseline RMSE - Test: 78375.26238032707


### Linear Regression

In [59]:
## Instantiate and fit a linear regression model to your training data
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [60]:
## Evaluate model on both training and testing data using RMSE
print(f"Train RMSE: {mean_squared_error(y_train, lr.predict(X_train))**0.5}")
print(f"Test RMSE: {mean_squared_error(y_test, lr.predict(X_test))**0.5}")

Train RMSE: 33950.579332858375
Test RMSE: 29118.462494524694


In [61]:
## Look at coefficients for each feature
lr.coef_

array([-1.24496932e-06,  1.17794933e+04,  9.70924167e+01,  1.15503973e+02,
        3.54102718e+01,  1.70946614e+04,  8.30807263e+03,  1.68432149e+00,
       -1.19278592e+00,  4.34924803e+01, -2.39792818e+03,  1.31384841e+04,
        2.26798972e+03,  3.30441215e+03, -1.76604192e+02,  2.56197962e+03,
        7.81909481e+03,  3.82782385e+01, -4.46057279e+03, -3.61465683e+03,
        1.65528036e-09,  2.72178200e+03, -7.24085682e+03,  1.57184298e+04,
       -9.01201952e+03,  9.27409772e+03,  2.03569720e+03, -4.99543361e+03,
        1.00859234e+04,  5.12954977e+03,  3.79411582e+04, -1.40153892e+04,
       -1.31944280e+04, -4.01917831e+03,  3.32088601e+04, -1.72466196e+04,
        7.83828554e+03,  1.24422662e+04, -4.61260082e+03,  6.76047450e+03])

In [62]:
## Let's make that a little easier to read
coef_df = pd.DataFrame({
    'column': X.columns,
    'coef'  : lr.coef_
})

In [63]:
coef_df.sort_values(by = 'coef', ascending = False)

,column,coef
30,House Style_2.5Fin,3.794116e+04
34,Garage Type_0,3.320886e+04
5,Exter Qual,1.709466e+04
23,Lot Config_CulDSac,1.571843e+04
11,Kitchen Qual,1.313848e+04
37,Garage Type_BuiltIn,1.244227e+04
1,Overall Qual,1.177949e+04
28,House Style_1.5Unf,1.008592e+04
25,Lot Config_FR3,9.274098e+03
6,Bsmt Qual,8.308073e+03


### Use your model to make predictions on the test dataset

In [66]:
## First we subset `df_test_full` to just the features we included in our model
X_kaggle = df_test[features]

In [67]:
X_kaggle.head()

,PID,Overall Qual,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Bsmt Qual,Total Bsmt SF,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,TotRms AbvGrd,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RL,MS Zoning_RM,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SLvl,Garage Type_0,Garage Type_2Types,Garage Type_Attchd,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2658,902301120,6,1910,1950,0.0,3,2,1020,908,1928,2,2,9,0,1910.0,1,1,440,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1
2718,905108090,5,1977,1977,0.0,3,4,1967,1967,1967,2,3,10,0,1977.0,3,2,580,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0
2414,528218130,7,2006,2006,0.0,4,4,654,664,1496,2,4,7,4,2006.0,2,2,426,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
1989,902207150,5,1923,2006,0.0,4,3,968,968,968,1,3,5,0,1935.0,1,2,480,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
625,535105100,6,1963,1963,247.0,3,4,1394,1394,1394,1,3,6,4,1963.0,2,2,514,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0


In [68]:
## Then we use the same model to predict on the test data, save predictions to a `SalePrice` column
X_kaggle['SalePrice'] = lr.predict(X_kaggle)

/Users/chrisjohnson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
X_kaggle.head()

,PID,Overall Qual,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Bsmt Qual,Total Bsmt SF,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,TotRms AbvGrd,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RL,MS Zoning_RM,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SLvl,Garage Type_0,Garage Type_2Types,Garage Type_Attchd,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2658,902301120,6,1910,1950,0.0,3,2,1020,908,1928,2,2,9,0,1910.0,1,1,440,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,126440.753673
2718,905108090,5,1977,1977,0.0,3,4,1967,1967,1967,2,3,10,0,1977.0,3,2,580,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,194166.237119
2414,528218130,7,2006,2006,0.0,4,4,654,664,1496,2,4,7,4,2006.0,2,2,426,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,207961.960459
1989,902207150,5,1923,2006,0.0,4,3,968,968,968,1,3,5,0,1935.0,1,2,480,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,135606.153040
625,535105100,6,1963,1963,247.0,3,4,1394,1394,1394,1,3,6,4,1963.0,2,2,514,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,190901.252590


In [70]:
output.shape

(2051, 1)

In [71]:
## Set output to be a dataframe with only the `SalePrice` column
output = X_kaggle[['SalePrice']]

In [73]:
output.shape

(878, 1)

In [74]:
## Save output to a csv
### (note: we're not using `index = False` here because we want our index `Id` to be a new column)
output.to_csv('./first_submission.csv')

In [75]:
output.shape

(878, 1)